# Hyperparameter tuning framework

- make sklearn compatible wrapper for econML xlearner

In [1]:
from econml.metalearners import XLearner
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

/Users/gabriellamessenger/Desktop/thesis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class EconMLWrapper:
    def __init__(self, est):
        self.est = est
        self.true_tau = None

    # Required by sklearn for cloning
    def get_params(self, deep=True):
        return {"est": self.est}

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

    # Fit with special wrapper
    def fit(self, X, y, score_y=None):
        Y = y[:, 0]
        W = y[:, 1]
        self.true_tau = score_y
        self.est.fit(Y, W, X=X)
        return self

    def predict(self, X):
        return self.est.effect(X)

    def score(self, X, y=None):
        preds = self.predict(X)
        return -np.mean((self.true_tau - preds) ** 2)